Task 1 = Generate a concise summary of the book "Crime and Punishment" limited to 20 pages.

In [1]:
import os
from langchain.embeddings import OpenAIEmbeddings

from langchain import OpenAI
os.environ["OPENAI_API_KEY"]  =  ""
llm= OpenAI()
embeddings  =   OpenAIEmbeddings()

c:\Users\umair\Documents\GitHub\CogentLabs-task\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\umair\Documents\GitHub\CogentLabs-task\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
from langchain.document_loaders import PyPDFLoader

# Load the book
loader = PyPDFLoader("./crime-and-punishment.pdf")
pages = loader.load_and_split()


In [76]:

# Combine the pages, and replace the tabs with spaces
text = ' '.join([page.page_content.replace('\t', ' ') for page in pages])

In [78]:
#remove the unnecessary content from the text like non-printable characters, extra spaces, etc.
import re
def clean_text(text):
   # Remove the specific phrase 'Free eBooks at Planet eBook.com' and surrounding whitespace
   cleaned_text = re.sub(r'\s*Free eBooks at Planet eBook\.com\s*', '', text, flags=re.DOTALL)
   # Remove extra spaces
   cleaned_text = re.sub(r' +', ' ', cleaned_text)
   # Remove non-printable characters, optionally preceded by 'Crime and Punishment' which you can see on whole pages in the PDF
   cleaned_text = re.sub(r'(Crime and Punishment)?[\x00-\x1F]', '', cleaned_text)
   # Replace newline characters with spaces
   cleaned_text = cleaned_text.replace('\n', ' ')
   # Remove spaces around hyphens
   cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text)
   return cleaned_text
clean_text=clean_text(text)

In [97]:
from langchain import OpenAI
llm = OpenAI()
Tokens = llm.get_num_tokens(clean_text)
print (f"We have total {Tokens} tokens in the book")

We have total 272188 tokens in the book


In [80]:

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
text_splitter = SemanticChunker(
   OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)
docs = text_splitter.create_documents([clean_text])

In [83]:
import numpy as np
import openai
def get_embeddings(text):
   response = openai.embeddings.create(
       model="text-embedding-3-small",
       input=text
   )
   return response.data
embeddings=get_embeddings([doc.page_content for doc in docs]
)

In [84]:

import pandas as pd
content_list = [doc.page_content for doc in docs]
df = pd.DataFrame(content_list, columns=['page_content'])
vectors = [embedding.embedding for embedding in embeddings]
array = np.array(vectors)
embeddings_series = pd.Series(list(array))
df['embeddings'] = embeddings_series

In [85]:

import numpy as np
import faiss
# Convert to float32 if not already
array = array.astype('float32') 
num_clusters = 50
# Vectors dimensionality
dimension = array.shape[1] 
# Train KMeans with Faiss
kmeans = faiss.Kmeans(dimension, num_clusters, niter=20, verbose=True)
kmeans.train(array)
# accessing created centroids
centroids = kmeans.centroids 
# Create a new index for the original dataset
index = faiss.IndexFlatL2(dimension)
# Add original dataset to the index
index.add(array) 


In [86]:

D, I = index.search(centroids, 1)

In [87]:
sorted_array = np.sort(I, axis=0)
sorted_array=sorted_array.flatten()
extracted_docs = [docs[i] for i in sorted_array]

In [90]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")


In [91]:
prompt = ChatPromptTemplate.from_template("""
You will be given different passages from a book one by one. Provide a summary of the following text. Your result must be detailed and concise format limited to 20 pages. When summarizing, directly dive into the narrative or descriptions from the text without using introductory phrases like 'In this passage'. Directly address the main events, characters, and themes, encapsulating the essence and significant details from the text in a flowing narrative. The goal is to present a unified view of the content, continuing the story seamlessly as if the passage naturally progresses into the summary.
Passage:
```{text}```
SUMMARY:
"""
)

In [92]:

chain= (
    prompt
   | model
   |StrOutputParser() )

In [93]:

from tqdm import tqdm
final_summary = ""

for doc in tqdm(extracted_docs, desc="Processing documents"):
   # Get the new summary.
   new_summary = chain.invoke({"text": doc.page_content})
   
   final_summary+=new_summary

Processing documents: 100%|██████████| 50/50 [03:03<00:00,  3.67s/it]


In [96]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        # Add book title
        self.set_font('Times', 'B', 16)
        self.cell(0, 10, 'Book Summary', 0, 1, 'C')
        self.ln(10)

    def footer(self):
        # Add page number
        self.set_y(-15)
        self.set_font('Times', 'I', 10)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

# Instantiate PDF object and add a page
pdf = PDF()
pdf.add_page()

# Set font for the main text
pdf.set_font("Times", size=12)



# Add the book summary text
pdf.multi_cell(0, 10, final_summary)

# Save the PDF to a file
pdf_output_path = "Crime_and_Punishment_summary.pdf"
pdf.output(pdf_output_path)


C:\Users\umair\AppData\Local\Temp\ipykernel_10304\3987256423.py:7: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Book Summary', 0, 1, 'C')
C:\Users\umair\AppData\Local\Temp\ipykernel_10304\3987256423.py:14: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')


Task 2 : Design a detailed prompt to create personalized study plans for students.


In [8]:
import os
from langchain import PromptTemplate
from langchain.llms import OpenAI
import pandas as pd

# Read the Excel file into a DataFrame
excel_file = "student_data.xlsx"
df = pd.read_excel(excel_file)
df

,Name,Field of study,Year of study,List of subjects,Preferred Learning Styles,Personal Objectives,Challenges,Extracurricular activities
0,umair tufail,Science,2,"Computer Science, Mathematics, History, English",Kinesthetic,"Improve grades, Prepare for exams","Time management, Understanding complex topics","Volunteering, Debate Club, Sports, Drama, Music"
1,Ayesha Siddiqui,Engineering,2,Science,"Auditory, Visual","Improve grades, Prepare for exams","Time management, Understanding complex topics","Volunteering, Drama, Debate Club"
2,Ahmed Khan,Arts,4,"Computer Science, Science, Mathematics","Auditory, Visual, Kinesthetic","Improve grades, Prepare for exams","Time management, Understanding complex topics","Debate Club, Music, Volunteering"
3,Fatima Ali,Medicine,3,"Science, Art, Mathematics",Auditory,"Improve grades, Prepare for exams","Time management, Understanding complex topics","Drama, Debate Club, Music, Volunteering"
4,Sana Shah,Business,1,"Mathematics, History, Art",Auditory,"Improve grades, Prepare for exams","Time management, Understanding complex topics",Sports
5,sumail,Arts,3,"Computer Science, History","Visual, Auditory, Kinesthetic","Improve grades, Prepare for exams","Time management, Understanding complex topics","Volunteering, Sports, Drama"
6,asad nadeem,Engineering,3,"Art, Mathematics, Science, Computer Science, E...","Auditory, Visual, Kinesthetic","Improve grades, Prepare for exams","Time management, Understanding complex topics","Drama, Debate Club, Volunteering"
7,hamza,Business,3,"Mathematics, Computer Science, Science, English","Visual, Kinesthetic, Auditory","Improve grades, Prepare for exams","Time management, Understanding complex topics","Sports, Drama, Volunteering"
8,kamran,Science,1,"Computer Science, Mathematics",Auditory,"Improve grades, Prepare for exams","Time management, Understanding complex topics","Volunteering, Music, Drama, Debate Club, Sports"
9,adeel,Medicine,4,"Computer Science, Mathematics, Science",Kinesthetic,"Improve grades, Prepare for exams","Time management, Understanding complex topics","Music, Drama, Volunteering, Sports, Debate Club"


In [3]:
# Define the input variables for the prompt
input_variables = ["Name", "Field of study", "Year of study", "List of subjects", "Preferred Learning Styles", "Personal Objectives", "Challenges", "Extracurricular activities"]

In [104]:
# Define the template for the prompt
custom_template = """
As an experienced academic advisor, your task is to create a highly personalized study plan tailored specifically for {Name}. 
You will be assisting a {Year of study}-year student who is currently pursuing a degree in {Field of study}.
Please gather the following information from the student:
- Subjects: {List of subjects}
- Learning Styles: {Preferred Learning Styles}
- Personal Objectives: {Personal Objectives}
- Challenges: {Challenges}
- Extracurricular Activities: {Extracurricular activities}
Based on above information, design a study plan that must includes the following:
1. Academic Performance: you must Provide strategies to improve performance in {List of subjects} individually.
2. Learning Styles: you must Recommend study methods tailored to {Preferred Learning Styles}.
3. Personal Objectives: you must Suggest ways to achieve {Personal Objectives}.
4. Challenges: you must Offer solutions to overcome {Challenges}.
5. Extracurricular Activities: you must Integrate study routines with {Extracurricular activities}.
Ensure that the study plan is well-structured, easy to follow, and supports the student's overall growth and success in their chosen field of study.
"""

In [105]:
# Create the PromptTemplate
study_plan_prompt = PromptTemplate(input_variables=input_variables, template=custom_template)

# Function to generate a study plan prompt for a specific student
def generate_study_plan_prompt(student_name):
    # Convert student name to lowercase
    student_name = student_name.lower()
    # Search for the student in the DataFrame (case-insensitive)
    student_data = df[df["Name"].str.lower() == student_name].iloc[0]
    # Create the prompt template using the student data
    study_plan_prompt_formatted_str = study_plan_prompt.format(**student_data)
    return study_plan_prompt_formatted_str

In [108]:
# Prompt the user to enter the name of the student
student_name = input("Enter the name of the student: ")

# Generate the study plan prompt for the specified student
prompt_for_student = generate_study_plan_prompt(student_name)

# Load the OpenAI API key from environment variables
openai_api_key = os.getenv("")

# Instantiate the OpenAI instance
llm = OpenAI(openai_api_key=openai_api_key,temperature=0.7,max_tokens=750, presence_penalty=0.3)

# Make a prediction
prediction = llm.predict(prompt_for_student)

# Print the prediction
from IPython.display import display

# Use a context manager to temporarily adjust the output settings
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(prediction)




Study Plan for Sumail:

1. Academic Performance:
a) Computer Science: 
- Create a study schedule and stick to it to ensure regular practice and understanding of concepts.
- Utilize online resources such as video tutorials, interactive quizzes, and coding challenges to supplement classroom learning.
- Attend office hours and seek help from the professor or teaching assistant for clarification on complex topics.
- Collaborate with classmates and form study groups to discuss and review course material.
- Practice coding on a regular basis to improve speed and accuracy.
- Take practice exams to assess understanding and identify areas of improvement.

b) History:
- Develop a note-taking system that works best for your learning style.
- Use visual aids such as diagrams, charts, and mind maps to organize and remember key information.
- Utilize flashcards to memorize important dates, events, and names.
- Discuss historical events with friends or family members to gain a deeper understanding.
